In [5]:
import numpy as np
import cv2
import os
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np

In [6]:
import tensorflow as tf
print(tf.version)

<module 'tensorflow._api.v2.version' from 'C:\\Users\\Sristi\\anaconda3\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>


In [3]:
TRAIN_DIR = 'E:/DrowsyBrowsy/archive/Eye Dataset/Train_Data'
TEST_DIR = 'E:/DrowsyBrowsy/archive/Eye Dataset/Test_Data'

In [4]:
print(TRAIN_DIR)

E:/DrowsyBrowsy/archive/Eye Dataset/Train_Data


In [5]:
def label_img(img):
    word_label = img.split('_')[4]
    if word_label[0] == '1': return [1,0] 
    elif word_label[0] == '0': return [0,1]
    
    #1 eyes are open
    #0 eyes are closed

In [6]:
X_test = []
Y_test = []


In [56]:
def create_train_data():
    X_train = []
    Y_train = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        resized_img = cv2.resize(img, (100, 100))
        X_train.append(np.array(resized_img))
        Y_train += [label]
        
    return X_train, Y_train


In [57]:
X_train,Y_train = create_train_data()

100%|██████████████████████████████████████████████████████████████████████████████████| 32623/32623 [00:20<00:00, 1600.24it/s]


In [58]:
X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)

In [59]:
len(X_train)

32623

In [60]:
len(Y_train)

32623

In [61]:
X_train.shape

(32623, 100, 100)

In [62]:
def create_test_data():
    X_test = []
    Y_test = []
    for img in tqdm(os.listdir(TEST_DIR)):
        label = label_img(img)
        path = os.path.join(TEST_DIR, img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        resized_img = cv2.resize(img, (100, 100))
        X_test.append(np.array(resized_img))
        Y_test += [label]
        
    return X_test, Y_test

In [63]:
X_test, Y_test = create_test_data()

100%|██████████████████████████████████████████████████████████████████████████████████| 14550/14550 [00:06<00:00, 2105.24it/s]


In [64]:
X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)

In [102]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

In [103]:
model = Sequential()

In [104]:
X_train.shape

(32623, 100, 100, 1)

In [105]:
Y_train.shape

(32623, 2)

In [106]:
X_train = X_train.reshape(32623, 100, 100, 1)

In [107]:
Y_train = Y_train.reshape(32623,2)

In [108]:
model.add(Conv2D(200, (3,3), input_shape=(100, 100, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(100, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.3))
model.add(Flatten())

model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [109]:
cp = ModelCheckpoint('model-best', verbose=0, save_best_only=True)
model.fit(X_train, Y_train, epochs = 6,callbacks=[cp], validation_split=0.2)

Epoch 1/6
816/816 [==============================] - 1624s 2s/step - loss: 3.5319 - accuracy: 0.8427 - val_loss: 0.5339 - val_accuracy: 0.6771
INFO:tensorflow:Assets written to: model-best\assets
Epoch 2/6
816/816 [==============================] - 1609s 2s/step - loss: 0.2323 - accuracy: 0.9149 - val_loss: 0.2158 - val_accuracy: 0.9350
INFO:tensorflow:Assets written to: model-best\assets
Epoch 3/6
816/816 [==============================] - 1502s 2s/step - loss: 0.1823 - accuracy: 0.9334 - val_loss: 0.2348 - val_accuracy: 0.9318
Epoch 4/6
816/816 [==============================] - 1485s 2s/step - loss: 0.3161 - accuracy: 0.8792 - val_loss: 0.6507 - val_accuracy: 0.6703
Epoch 5/6
816/816 [==============================] - 1452s 2s/step - loss: 0.5594 - accuracy: 0.7527 - val_loss: 0.6549 - val_accuracy: 0.6703
Epoch 6/6
816/816 [==============================] - 1466s 2s/step - loss: 0.5562 - accuracy: 0.7556 - val_loss: 0.6537 - val_accuracy: 0.6703


In [7]:
eyeCascade = cv2.CascadeClassifier("haarcascade_eye.xml")
leyeCascade = cv2.CascadeClassifier("haarcascade_lefteye_2splits.xml")
reyeCascade = cv2.CascadeClassifier("haarcascade_righteye_2splits.xml")

In [8]:
import cv2

In [9]:
faceCascade= cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [10]:
model = load_model('model-best')

In [11]:
model1 = load_model('yaw-yaw')

In [12]:
from time import time

In [47]:
cap=cv2.VideoCapture(0)
cap.set(10,100)
blink_count = 0
blink_status = 0
t1 = t1_ = time()
wake = 1
sleep = 0
yawn_count = 0
yawn_status = 0
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(imgGray,1.1,4)
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        gray_face=imgGray[y:y+h,x:x+w]
        color_face=img[y:y+h,x:x+w]
        resized_image = cv2.resize(gray_face, (100, 100))
        reshaped_face = np.reshape(resized_image, (1, 100, 100, 1))
        predyawn = model1.predict(reshaped_face)
        
        eye = eyeCascade.detectMultiScale(gray_face,1.3,5)
        leye = leyeCascade.detectMultiScale(gray_face,1.3,5)
        reye = reyeCascade.detectMultiScale(gray_face,1.3,5)
        
        for (a,b,c,d) in leye:
            cv2.rectangle(color_face,(a,b),(a+c,b+d),(0,255,0),2)
            l_eye = color_face[b:b+d,a:a+c]
            resized_image = cv2.resize(l_eye, (100, 100))
            l_eye = cv2.cvtColor(resized_image,cv2.COLOR_BGR2GRAY)
            #normalized_image = l_eye/255
            reshaped_eye = np.reshape(l_eye, (1, 100, 100, 1))
            lpred = model.predict(reshaped_eye)
                     
        for (a,b,c,d) in reye:
            cv2.rectangle(color_face,(a,b),(a+c,b+d),(0,255,0),2)
            r_eye = color_face[b:b+d,a:a+c]
            resized_image = cv2.resize(r_eye, (100, 100))
            r_eye = cv2.cvtColor(resized_image,cv2.COLOR_BGR2GRAY)
            #normalized_image = l_eye/255
            reshaped_eye = np.reshape(r_eye, (1, 100, 100, 1))
            rpred = model.predict(reshaped_eye)
            
        
        if rpred[0][0] < rpred[0][1] and lpred[0][0] < lpred[0][1]:
            if blink_status == 0:
                blink_count += 1
                blink_status = 1
        else: 
            blink_status = 0
 
 #Eye blink frequency 
    t2 = time()
    count_down = t2 - t1
    #10 count = 4,   count n= 1
    if count_down >= 15:
        if blink_count > 4:
            wake = 1
        else:
            wake = 0     
        t1 = time()
        blink_count = 0
        
    elapsed_time = t2 - t1_ 
    if elapsed_time >= 10:
        if yawn_count > 1:
            sleep = 1
        elif yawn_count == 1:
            sleep = 0.5
        else:
            sleep = 0
        t1_ = time()
        yawn_count = 0    
     
    if predyawn[0][0] < predyawn[0][1]:
        if yawn_status == 0:
            yawn_count += 1
            yawn_status = 1
    else: 
        yawn_status = 0
        
    cv2.putText(img,'Yawns = ' + str(yawn_count) + ', Blinks = ' + str(blink_count),(x, y - 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        # Awake # Very Awake 
    if sleep == 1: # red orange yellow
        cv2.putText(img,"Fatigued",(x+50, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    elif not wake and sleep == 0.5:
        cv2.putText(img,"Semi Fatigued",(x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (57, 143, 235), 2)
    elif wake and sleep == 0.5:
        cv2.putText(img,"Drowsy",(x+50, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (58, 238, 238), 2)
    elif wake:
        cv2.putText(img,"Awake",(x+50, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (67, 146, 52), 2)
    else:
        cv2.putText(img,"Semi Drowsy",(x+50, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (57, 235, 143), 2)
   
            
    cv2.imshow("Result", img)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()



In [14]:
cap.release()
cv2.destroyAllWindows()